In [1]:
import pandas as pd
import numpy as np

# ==========================================
# 1. Load Data
# ==========================================
# Loading raw CSVs from the Bronze layer.
fact = pd.read_csv('data/bronze_fact_sales.csv')
dim_product = pd.read_csv('data/bronze_dim_product.csv')
dim_location = pd.read_csv('data/bronze_dim_location.csv')


In [2]:

# ==========================================
# 2. Standardize & Temporal Engineering
# ==========================================
# Cleaning dates and creating Year, Month, and Day columns for precise filtering.
fact['date'] = pd.to_datetime(fact['date'], errors='coerce')
fact['year'] = fact['date'].dt.year
fact['month'] = fact['date'].dt.month
fact['day'] = fact['date'].dt.day
fact['discount_amount'] = fact['discount_amount'].fillna(0)
fact['is_returned'] = fact['is_returned'].fillna(False).astype(bool)
fact.head()


,transaction_id,date,location_id,product_id,quantity,unit_price_sold,gross_sales_amt,discount_amount,is_returned,return_reason,year,month,day
0,0816bdbed81a,2019-01-09,L001_WEB,P095,1,105.37,105.37,0.0,False,NaN,2019,1,9
1,b4bc21763e5e,2019-01-04,S023,P042,4,52.71,210.84,54.4,True,Allergic Reaction,2019,1,4
2,9c0c300e4dd6,2019-01-22,S014,P097,1,88.62,88.62,0.0,False,NaN,2019,1,22
3,c6cd846ff755,2019-01-23,S009,P063,1,80.51,80.51,0.0,False,NaN,2019,1,23
4,3bef0fc8390b,2019-01-20,L001_WEB,P031,2,37.12,74.24,0.0,False,NaN,2019,1,20


In [3]:
# ==========================================
# 3. Enrich & Create Gold Layer
# ==========================================
# Merging dimensions and aggregating data to daily store performance.
fact_enriched = fact.merge(dim_product, on='product_id', how='left')\
                    .merge(dim_location, on='location_id', how='left')

fact_enriched


,transaction_id,date,location_id,product_id,quantity,unit_price_sold,gross_sales_amt,discount_amount,is_returned,return_reason,...,month,day,product_name,category,sub_category,price,location_type,city,state,region
0,0816bdbed81a,2019-01-09,L001_WEB,P095,1,105.37,105.37,0.00,False,NaN,...,1,9,Matte Conditioner,Haircare,Conditioner,105.37,Web,NaN,NaN,Digital
1,b4bc21763e5e,2019-01-04,S023,P042,4,52.71,210.84,54.40,True,Allergic Reaction,...,1,4,Volumizing Body Mist,Fragrance,Body Mist,66.31,Retail Store,Thomasberg,MO,Midwest
2,9c0c300e4dd6,2019-01-22,S014,P097,1,88.62,88.62,0.00,False,NaN,...,1,22,Hydrating Lipstick,Makeup,Lipstick,88.62,Retail Store,Port Craig,CO,Southeast
3,c6cd846ff755,2019-01-23,S009,P063,1,80.51,80.51,0.00,False,NaN,...,1,23,Hydrating Eau de Parfum,Fragrance,Eau de Parfum,80.51,Retail Store,South Colinstad,CT,Northeast
4,3bef0fc8390b,2019-01-20,L001_WEB,P031,2,37.12,74.24,0.00,False,NaN,...,1,20,Sheer Rollerball,Fragrance,Rollerball,37.12,Web,NaN,NaN,Digital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,4e313985c8be,2019-01-30,L001_WEB,P127,1,98.44,98.44,0.00,False,NaN,...,1,30,Glowing Styling Cream,Haircare,Styling Cream,98.44,Web,NaN,NaN,Digital
2995,0aa73b18f7c7,2019-01-12,L001_WEB,P051,2,56.88,113.76,8.40,False,NaN,...,1,12,Hydrating Moisturizer,Skincare,Moisturizer,61.08,Web,NaN,NaN,Digital
2996,bfab86001a94,2019-01-01,S020,P096,1,66.92,66.92,38.88,False,NaN,...,1,1,Glowing Hair Oil,Haircare,Hair Oil,105.80,Retail Store,East Lydiamouth,WI,Midwest
2997,542e04e34450,2019-01-28,S012,P119,1,62.35,62.35,0.00,False,NaN,...,1,28,Repairing Shampoo,Haircare,Shampoo,62.35,Retail Store,Barbaraland,MP,Southeast


In [4]:
# This ensures fact_enriched has the column before you try to filter it
fact_enriched['discount_percentage'] = (fact_enriched['discount_amount'] / fact_enriched['gross_sales_amt'].replace(0, 1)) * 100
fact_enriched.head()

,transaction_id,date,location_id,product_id,quantity,unit_price_sold,gross_sales_amt,discount_amount,is_returned,return_reason,...,day,product_name,category,sub_category,price,location_type,city,state,region,discount_percentage
0,0816bdbed81a,2019-01-09,L001_WEB,P095,1,105.37,105.37,0.0,False,NaN,...,9,Matte Conditioner,Haircare,Conditioner,105.37,Web,NaN,NaN,Digital,0.000000
1,b4bc21763e5e,2019-01-04,S023,P042,4,52.71,210.84,54.4,True,Allergic Reaction,...,4,Volumizing Body Mist,Fragrance,Body Mist,66.31,Retail Store,Thomasberg,MO,Midwest,25.801556
2,9c0c300e4dd6,2019-01-22,S014,P097,1,88.62,88.62,0.0,False,NaN,...,22,Hydrating Lipstick,Makeup,Lipstick,88.62,Retail Store,Port Craig,CO,Southeast,0.000000
3,c6cd846ff755,2019-01-23,S009,P063,1,80.51,80.51,0.0,False,NaN,...,23,Hydrating Eau de Parfum,Fragrance,Eau de Parfum,80.51,Retail Store,South Colinstad,CT,Northeast,0.000000
4,3bef0fc8390b,2019-01-20,L001_WEB,P031,2,37.12,74.24,0.0,False,NaN,...,20,Sheer Rollerball,Fragrance,Rollerball,37.12,Web,NaN,NaN,Digital,0.000000


In [5]:

# Save enriched data for drill-down tool
fact_enriched.to_csv('fact_enriched.csv', index=False)


In [6]:

# 'fact_enriched' is your merged dataframe
# 1. First, group the data by your dimensions (Year, Month, Day, Location)
gold_daily_performance = fact_enriched.groupby(['year', 'month', 'day', 'location_id']).agg({
    'gross_sales_amt': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum',
    'is_returned': 'sum'
}).reset_index()

gold_daily_performance.head()


,year,month,day,location_id,gross_sales_amt,discount_amount,quantity,is_returned
0,2019,1,1,L001_WEB,3852.90,48.11,58,0
1,2019,1,1,S001,597.18,0.00,6,0
2,2019,1,1,S002,269.35,29.28,3,0
3,2019,1,1,S003,111.95,0.00,1,0
4,2019,1,1,S004,344.08,0.00,4,0


In [7]:


# 2. THEN calculate the percentage on the aggregated result
# Use .replace(0, 1) to prevent division by zero errors
gold_daily_performance['discount_percentage'] = (
    gold_daily_performance['discount_amount'] / 
    gold_daily_performance['gross_sales_amt'].replace(0, 1)
) * 100

gold_daily_performance.head()


,year,month,day,location_id,gross_sales_amt,discount_amount,quantity,is_returned,discount_percentage
0,2019,1,1,L001_WEB,3852.90,48.11,58,0,1.248670
1,2019,1,1,S001,597.18,0.00,6,0,0.000000
2,2019,1,1,S002,269.35,29.28,3,0,10.870614
3,2019,1,1,S003,111.95,0.00,1,0,0.000000
4,2019,1,1,S004,344.08,0.00,4,0,0.000000


In [8]:

# 3. Check for any non-zero values
# print("Sample of calculated percentages:")
gold_daily_performance[gold_daily_performance['discount_percentage'] > 0].head()


,year,month,day,location_id,gross_sales_amt,discount_amount,quantity,is_returned,discount_percentage
0,2019,1,1,L001_WEB,3852.90,48.11,58,0,1.248670
2,2019,1,1,S002,269.35,29.28,3,0,10.870614
5,2019,1,1,S005,274.05,7.77,5,0,2.835249
6,2019,1,1,S006,269.90,126.99,5,0,47.050760
12,2019,1,1,S013,501.25,7.80,5,0,1.556110


In [9]:


# Save the final Gold table
gold_daily_performance.to_csv('gold_daily_performance.csv', index=False)
print("Data Engineering Complete: Files saved as 'fact_enriched.csv' and 'gold_daily_performance.csv'")


Data Engineering Complete: Files saved as 'fact_enriched.csv' and 'gold_daily_performance.csv'


In [10]:

# Save this directly to your environment
sops = {
    "SOP-QA-001.txt": "SOP-QA-001: Product Adverse Reaction Protocol. Action: If 2+ incidents in 48h, mark SKU 'Do Not Sell' and submit Level 1 Quality Alert.",
    "SOP-FIN-003.txt": "SOP-FIN-003: Pricing and Discount Policy. Action: Unauthorized discounts >15% require Regional Director approval and trigger a financial audit.",
    "SOP-LOG-002.txt": "SOP-LOG-002: E-commerce Fulfillment Standards. Action: For late returns during Peak, waive shipping fees and offer 10% courtesy coupon.",
    "SOP-OPS-004.txt": "SOP-OPS-004: Inventory OSA Protocol. Action: If Top 20 SKU is empty, perform backroom check and adjust system inventory to ZERO."
}

for filename, content in sops.items():
    with open(f'{filename}', 'w') as f:
        f.write(content)
print("SOP text files created successfully.")

SOP text files created successfully.
